# Stable Diffusion notebook by [@alexpopo](https://github.com/alexpopo/alex_stable_diffusion_webui)

This let's you generate images with CompVis/Stability [Stable Diffusion](https://github.com/CompVis/stable-diffusion) with bonus [KLMS sampling](https://github.com/crowsonkb/k-diffusion.git) from [@RiversHaveWings](https://twitter.com/RiversHaveWings)

You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/runwayml/stable-diffusion-v1-5).

Other Stable Diffusion tools:
* [Deforum Stable Diffusion](https://colab.research.google.com/github/deforum/stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb) by [deforum_art](https://twitter.com/deforum_art) team
* [HuggingFace space for Stable Diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
* [Stable Diffusion Interpolation](https://colab.research.google.com/drive/1EHZtFjQoRr-bns1It5mTcOVyZzZD9bBc?usp=sharing) by [@ygantigravity](https://twitter.com/ygantigravity)
* [Stable Diffusion with diffusers](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=zHkHsdtnry57)
* [Prompt-space Interpolation](https://colab.research.google.com/drive/1gj8_EPyntZuaiQuzYOzog5YKA74esoB3#scrollTo=3g5rSm43STsW) by [@DigThatData](https://twitter.com/DigThatData)
* [Stable Diffusion Lite](https://colab.research.google.com/drive/1cl3d84B5AXepycKAfCba77faUFBRGY2O?usp=sharing#scrollTo=Rn06QzsdsICR) by [@future__art](https://twitter.com/future__art)
* [Stable Diffusion web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui)
* More to come!!

If you're looking for more Ai art tools check here [Ai generative art tools list](https://pharmapsychotic.com/tools.html).

if you encounter any issues, feel free to discuss them.[Discord Support](https://discord.com/channels/1002292111942635562)

In [1]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d770efba-41d3-4fa0-bc15-e76ccc3f1ce6)


In [ ]:
#@title Mount Google Drive

#@markdown ### 谷歌云端硬盘：仅第一次需要下载模型文件，生成的图片样本将保存在谷歌云端硬盘
#@markdown ### 简单便捷方式：每次重新连接CoLab需要重新下载所有模型文件，不需要云硬盘

import os
import time
from google.colab import drive
drive.mount('/content/gdrive')

time.sleep(1)

if os.path.exists("/content/gdrive/MyDrive/"):
  print('Gdrive connected, using Gdrive storage ...')
  print("谷歌云端硬盘方式")
else:
  print('[1;31mGdrive not connected, using colab storage ...')
  print("简单便捷方式")

Mounted at /content/gdrive
Gdrive connected, using Gdrive storage ...
谷歌云端硬盘方式


In [ ]:
#@title Learning Environment Settings

#@markdown ### Please recompile "Processing Function Code" after changing settings
# Debug Flag, Default is False
CLEAR_SCREEN_SETTING = True #@param {type:"boolean"}

# Download Model Flag, Default is False
DOWNLOAD_MODEL_SETTING = True #@param {type:"boolean"}

# User Data Dir Flag, Default is False
USER_DATA_DIR_SETTING = True #@param {type:"boolean"}

outputs_path = "/content/SDWebUI"
!mkdir -p $outputs_path
!cd $outputs_path
print("Colab Learning Environment Path: ", outputs_path)

Colab Learning Environment Path:  /content/SDWebUI


In [ ]:
#@title Processing Function Code

import os
import time
from IPython.display import clear_output

# Clear Current Screen Output Infomation Function
def clear_screen(delay_time=3):
  """
  Note: refer to CLEAR_SCREEN_SETTING
  """
  if CLEAR_SCREEN_SETTING==False:
    return

  time.sleep(delay_time)
  clear_output()
  print("Successful Execution.")


# Download Stable Diffusion Model Checkpoint Function
def download_sd_model(model_version="Default"):
  """
  Note: refer to CLEAR_SCREEN_SETTING
  """
  """
    parameter：
    model_version = [ "Default", "1.5-pruned", "V2.1-768px", "Anything-v4.5", "Midjourney-v4-diffusion"]

    return:
     model_param
  """

  model_param = ""
  model_save_path = ""

  if os.path.exists("/content/gdrive/MyDrive/"):
    model_save_path = "/content/gdrive/MyDrive/AI/models/Stable-diffusion"
    %mkdir -p $model_save_path
    %cd $model_save_path
  else:
    model_save_path = "/content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion"
    %mkdir -p $model_save_path
    %cd $model_save_path

  if DOWNLOAD_MODEL_SETTING==False:
    model_param = model_save_path + '/' + "sd-v1-4.ckpt"
    return model_param

  if os.path.exists("/content/gdrive/MyDrive/"):
    if not os.path.exists("sd-v1-4.ckpt") and model_version=="Default":
      !wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/sd-v1-4.ckpt
    elif not os.path.exists("v1-5-pruned-emaonly.ckpt") and model_version=="1.5-pruned":
      !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt
    elif not os.path.exists("v2-1_768-ema-pruned.ckpt") and model_version=="V2.1-768px":
      !wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/v2-1_768-ema-pruned.ckpt
    elif not os.path.exists("anything-v4.5-pruned.ckpt") and model_version=="Anything-v4.5":
      !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/anything-v4.5-pruned.ckpt
      !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/anything-v4.5-pruned.vae.pt
    elif not os.path.exists("mdjrny-v4.ckpt") and model_version=="Midjourney-v4-diffusion":
      !wget https://huggingface.co/ckpt/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/mdjrny-v4.ckpt
  else:
    if not os.path.exists("sd-v1-4.ckpt") and model_version=="Default":
      !wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt
    elif not os.path.exists("v1-5-pruned-emaonly.ckpt") and model_version=="1.5-pruned":
      !wget https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/v1-5-pruned-emaonly.ckpt
    elif not os.path.exists("v2-1_768-ema-pruned.ckpt") and model_version=="V2.1-768px":
      !wget https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.ckpt
    elif not os.path.exists("anything-v4.5-pruned.ckpt") and model_version=="Anything-v4.5":
      !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.5-pruned.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-pruned.ckpt
      !wget https://huggingface.co/andite/anything-v4.0/resolve/main/anything-v4.0.vae.pt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/anything-v4.5-pruned.vae.pt
    elif not os.path.exists("mdjrny-v4.ckpt") and model_version=="Midjourney-v4-diffusion":
      !wget https://huggingface.co/ckpt/midjourney-v4-diffusion/resolve/main/mdjrny-v4.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/mdjrny-v4.ckpt

  if model_version=="Default":
    model_param = model_save_path + '/' + "sd-v1-4.ckpt"
  elif model_version=="1.5-pruned":
    model_param = model_save_path + '/' + "v1-5-pruned-emaonly.ckpt"
  elif model_version=="V2.1-768px":
    model_param = model_save_path + '/' + "v2-1_768-ema-pruned.ckpt"
  elif model_version=="Anything-v4.5":
    model_param = model_save_path + '/' + "anything-v4.5-pruned.ckpt"
  elif model_version=="Midjourney-v4-diffusion":
    model_param = model_save_path + '/' + "mdjrny-v4.ckpt"
  else:
    model_param = model_save_path + '/' + "model.ckpt"

  if model_version=="V2.1-768px":
    %cd $model_save_path
    !git clone https://github.com/Stability-AI/stablediffusion.git $outputs_path/stablediffusion
    !mv -f /content/SDWebUI/stablediffusion/configs/stable-diffusion/v2-inference-v.yaml v2-1_768-ema-pruned.yaml
    !rm -r /content/SDWebUI/stablediffusion

  return model_param


# Setting User Data Dir Function
def setting_user_data_dir(set_data_dir):
  """
  Note: refer to USER_DATA_DIR_SETTING
  """
  if USER_DATA_DIR_SETTING==False:
    data_dir_param = ""
    return data_dir_param

  if set_data_dir == "":
    data_dir_param = "/content/gdrive/MyDrive/AI"
  else:
    data_dir_param = set_data_dir

  !mkdir -p $data_dir_param
  !mkdir -p $data_dir_param/extensions
  !mkdir -p $data_dir_param/models
  !mkdir -p $data_dir_param/models/Codeformer
  !mkdir -p $data_dir_param/models/ESRGAN
  !mkdir -p $data_dir_param/models/GFPGAN
  !mkdir -p $data_dir_param/models/LDSR
  !mkdir -p $data_dir_param/models/Lora
  !mkdir -p $data_dir_param/models/Stable-diffusion
  !mkdir -p $data_dir_param/models/SwinIR
  !mkdir -p $data_dir_param/models/VAE-approx
  !mkdir -p $data_dir_param/models/hypernetworks

  return data_dir_param



In [ ]:
#@title First Base Requirements

# 在尝试安装之前，请确保满足所有必需的依赖项
print("Check Common Software Versions ...")
!git --version
print()
print("git-v2.12及之后版本集成了git-lfs")
print("-"*80)
!apt --version
print("-"*80)
!python3 --version
print("-"*80)
!wget --version | grep "^GNU Wget"
print("-"*80)

print('Installing base requirements...')

# Debian-based:
!sudo apt install python3-venv

clear_screen()

Successful Execution.


In [ ]:
#@title Next Base Requirements

import os

# Clone WebUI Repo Code And Download SD Model
# Clone WebUI Repo Code
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git $outputs_path/stable-diffusion-webui

"""
Git LFS Details
SHA256: fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556
Pointer size: 135 Bytes
Size of remote file: 4.27 GB
"""
# 下载模型文件"sd-v1-4.ckpt"并进行hash校验，检查是否下载完整
if os.path.exists("/content/gdrive/MyDrive/"):
  %mkdir -p /content/gdrive/MyDrive/AI/models/Stable-diffusion
  %cd /content/gdrive/MyDrive/AI/models/Stable-diffusion
  %rm -r /content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion
  !ln -s /content/gdrive/MyDrive/AI/models/Stable-diffusion /content/SDWebUI/stable-diffusion-webui/models

  %mkdir -p /content/gdrive/MyDrive/AI/outputs
  !ln -s /content/gdrive/MyDrive/AI/outputs /content/SDWebUI/stable-diffusion-webui

  if not os.path.exists("sd-v1-4.ckpt"):
    download_sd_model()
    !sha256sum sd-v1-4.ckpt
else:
  download_sd_model()
  !sha256sum $outputs_path/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt

Cloning into '/content/SDWebUI/stable-diffusion-webui'...
remote: Enumerating objects: 16079, done.
remote: Total 16079 (delta 0), reused 0 (delta 0), pack-reused 16079
Receiving objects: 100% (16079/16079), 27.20 MiB | 39.78 MiB/s, done.
Resolving deltas: 100% (11292/11292), done.
/content/gdrive/MyDrive/AI/models/Stable-diffusion


In [ ]:
#@title Automatic Installation (Take a ten minute break with a cup of tea)

%cd $outputs_path/stable-diffusion-webui

!sed -i 's@#install_dir="/home/$(whoami)"@install_dir="/content/SDWebUI"@' $outputs_path/stable-diffusion-webui/webui-user.sh
!sed -i "s@can_run_as_root=0@can_run_as_root=1@" $outputs_path/stable-diffusion-webui/webui.sh

# [Bug]: fastapi 0.91 app.add_middleware #7714
# [Fix]: Roll back the previous version #7717
!sed -i "/httpcore<=0.15/afastapi==0.90.1" $outputs_path/stable-diffusion-webui/requirements_versions.txt

!bash webui.sh

clear_screen(1)

Successful Execution.


In [ ]:
#@title WebUI自动启动成功会有下面提示，停止上面的自动运行，安装'xformers'模块
# Running on local URL:  http://127.0.0.1:7860
# To create a public link, set `share=True` in `launch()`.

# Launching Web UI with arguments: No module 'xformers' Issues.
# fix bug: install xformers-0.0.16-cp38-cp38-manylinux2014_x86_64.whl
%cd /content/SDWebUI/stable-diffusion-webui

!source venv/bin/activate; python3 -m pip install xformers==0.0.16; python3 -m pip list | grep -e "xformers"

# [Bug] FileNotFoundError: [Errno 2] No such file or directory: '.../models/VAE-approx/model.pt'
# [Fix]: Set Data Dir Problem, Copy missing model files.
!mkdir -p /content/gdrive/MyDrive/AI/models/VAE-approx
!mv /content/SDWebUI/stable-diffusion-webui/models/VAE-approx/model.pt /content/gdrive/MyDrive/AI/models/VAE-approx/model.pt

clear_screen()

Successful Execution.


### Prompt Demo

* green sapling rowing out of ground, mud, dirt, grass, high quality, photorealistic, sharp focus, depth of field

* Steps: 20, Sampler: Euler a, CFG scale: 12, Seed: 1441787169, Size: 512x512, Model hash: fe4efff1e1, Model: sd-v1-4

# --------------------------------------------------------------------------------------------------------------

* anime fine details portrait of joyful Deku in front of cyberpunk moder city landscape on the background deep bokeh, close-up view, anime masterpiece by Studio Ghibli, 4k, sharp high quality anime
* Steps: 25, Sampler: DPM++ SDE Karras, CFG scale: 7.5, Seed: 1750804985, Size: 512x768, Model hash: e4b17ce185, Model: anything-v4.5-pruned

# --------------------------------------------------------------------------------------------------------------

* woman with dress made of flowing water waves splashing, by Alberto Seveso
* Steps: 99, Sampler: Euler a, CFG scale: 7.5, Seed: 2571042388, Size: 512x1024, Model hash: 5d5ad06cc2, Model: mdjrny-v4

In [ ]:
#@title Restart Stable-Diffusion
import os

#@markdown Model Download/Load Checkpoint
Set_Model_Version = "Midjourney-v4-diffusion" #@param [ "Default", "1.5-pruned", "V2.1-768px", "Anything-v4.5", "Midjourney-v4-diffusion"]
model_param = download_sd_model(Set_Model_Version)

#@markdown Default User Data Output : /content/gdrive/MyDrive/AI (optional)
Set_User_Data_Dir = "" #@param {type:"string"}
ddir_param = setting_user_data_dir(Set_User_Data_Dir)

#@markdown Add credentials to your Gradio interface (optional)
User = "alexpopo" #@param {type:"string"}
Password= "123456" #@param {type:"string"}

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

%cd $outputs_path/stable-diffusion-webui

try:
  if os.path.isfile(model_param):
    !source venv/bin/activate; python3 webui.py --share --xformers $auth --theme="dark" --ckpt "$model_param" --data-dir $ddir_param --disable-console-progressbars
  else:
    !source venv/bin/activate; python3 webui.py --share --xformers $auth --theme="light" --ckpt-dir "$model_param" --data-dir $ddir_param --disable-console-progressbars
except:
  !source venv/bin/activate; python3 webui.py --share --xformers $auth